# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235387757915                   -0.50    7.0         
  2   -7.250422632806       -1.82       -1.40    1.0   7.26ms
  3   -7.251185889656       -3.12       -2.18    1.0   7.11ms
  4   -7.251197654827       -4.93       -2.07    2.0   9.18ms
  5   -7.251336538873       -3.86       -2.77    1.0   7.07ms
  6   -7.251338582936       -5.69       -3.42    1.0   7.32ms
  7   -7.251338739296       -6.81       -3.62    2.0   8.75ms
  8   -7.251338794579       -7.26       -4.14    1.0   7.21ms
  9   -7.251338798242       -8.44       -4.74    1.0   7.33ms
 10   -7.251338798655       -9.38       -5.26    2.0   8.71ms
 11   -7.251338798697      -10.37       -5.55    2.0   9.13ms
 12   -7.251338798703      -11.22       -6.05    1.0   7.50ms
 13   -7.251338798704      -11.94       -6.36    2.0   52.5ms
 14   -7.251338798705      -12.68       -6.87    2.0   9.15ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04777730503317939
[ Info: Arnoldi iteration step 2: normres = 0.5010289260544466
[ Info: Arnoldi iteration step 3: normres = 0.9666603773281056
[ Info: Arnoldi iteration step 4: normres = 0.3751598582505789
[ Info: Arnoldi iteration step 5: normres = 0.3907628437219316
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.00e-02, 5.97e-02, 3.14e-01, 2.15e-01, 4.24e-02)
[ Info: Arnoldi iteration step 6: normres = 0.33587403082157835
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.42e-02, 8.77e-02, 2.98e-01, 1.06e-01, 3.29e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09919318003776179
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.93e-04, 1.07e-02, 2.52e-02, 4.55e-02, 7.52e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10409093405136391
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.17e-